This shows how to merge (source) DEM tiles using the subroutine `get_dem_tile_paths` and then into a VRT using [`gdal.BuildVRT`](https://gis.stackexchange.com/a/314580). This can be completed localizing the data, or if urls point to geotiff files as is the case with `glo_30`, using the urls directly. Because `stitch_dem` places all rasters in memory, `stitch_dem` is not applicable to larger areas e.g. across the continental United States. `get_dem_tile_paths`  allows for handling the tiles directly without transformations.

**Note**: `get_dem_tile_paths` demonstration below does not perform any transformations e.g. conversion from geoid to ellipsoidal height. Such tile transformations are in the domain of `stitch_dem`, but can be adapted to the tiles directly using various functions in this library.

**Warning**: `get_dem_tile_paths` does not perform any dateline wrapping/unwrapping of the rasters. However, the function does wrap the extent and returns all tile paths colocated in an input extent. Thus, if an extent passes across the dateline, then it will get tiles at opposite hemispheres. In order to format into a contiguous raster, the tiles will require translation of their geotransform. This wrapping/unwrapping of the rasters is done automatically in `stitch_dem`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dem_stitcher.stitcher import get_dem_tile_paths, get_overlapping_dem_tiles
from osgeo import gdal
import rasterio

In [3]:
dem_name = 'glo_30'

There is an option to localize the data.

In [4]:
dem_tile_paths = get_dem_tile_paths(bounds = [-121.5, 34.95, -120.2, 36.25], 
                                    dem_name=dem_name, 
                                    localize_tiles_to_gtiff=(False if dem_name == 'glo_30' else True), 
                                    tile_dir=f'{dem_name}_tiles')
dem_tile_paths

['https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N34_00_W121_00_DEM/Copernicus_DSM_COG_10_N34_00_W121_00_DEM.tif',
 'https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N35_00_W121_00_DEM/Copernicus_DSM_COG_10_N35_00_W121_00_DEM.tif',
 'https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N35_00_W122_00_DEM/Copernicus_DSM_COG_10_N35_00_W122_00_DEM.tif',
 'https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N36_00_W121_00_DEM/Copernicus_DSM_COG_10_N36_00_W121_00_DEM.tif',
 'https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N36_00_W122_00_DEM/Copernicus_DSM_COG_10_N36_00_W122_00_DEM.tif']

In [5]:
vrt_path = f'{dem_name}.vrt'

In [6]:
ds = gdal.BuildVRT(vrt_path, dem_tile_paths)
ds = None

Note the bounds contain the original requested area (as the tiles cover the entire area).

In [7]:
with rasterio.open(vrt_path) as ds:
    bounds = ds.bounds
bounds

BoundingBox(left=-122.00013888888888, bottom=34.00013888888889, right=-120.00013888888888, top=37.00013888888889)